## LSTM Forecasting with PyTorch
This notebook implements an LSTM model to forecast page views using PyTorch.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Prepare data
df = pd.read_csv('wiki_traffic_cleaned.csv')
page = df['Page'].unique()[0]
series = df[df['Page'] == page]['Views'].values.reshape(-1,1)

scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series)

# Sequence data
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length):
        xs.append(data[i:i+seq_length])
        ys.append(data[i+seq_length])
    return np.array(xs), np.array(ys)

seq_length = 30
X, y = create_sequences(series_scaled, seq_length)

X_train, y_train = torch.FloatTensor(X[:-30]), torch.FloatTensor(y[:-30])
X_test, y_test = torch.FloatTensor(X[-30:]), torch.FloatTensor(y[-30:])

# Model
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        return out

model = LSTM()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train
for epoch in range(30):
    model.train()
    optimizer.zero_grad()
    out = model(X_train)
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1)%10==0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# Predict
model.eval()
preds = model(X_test).detach().numpy()
preds = scaler.inverse_transform(preds)
actual = scaler.inverse_transform(y_test.detach().numpy())

# Plot
plt.figure(figsize=(10,4))
plt.plot(range(30), actual, label='Actual')
plt.plot(range(30), preds, label='LSTM Forecast')
plt.legend()
plt.show()